In [289]:
print('Lets build the model :)')
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
#In this modeling approach we are taking 3 previous and trying to predict the 4th word in the sequance

Lets build the model :)


In [290]:
# reading the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [291]:
len(words)

32033

In [292]:
# building the mappings of characters to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [293]:
#building the dataset
block_size = 3 #context length how many characters do we take to predict the next one
X,Y = [],[]
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

        

In [294]:
g = torch.Generator().manual_seed(219384384)
#creating a lookup table for the character embeddings
C = torch.randn((27,2), generator = g)
# Constructiong the hidden layer
#The number of inputs to this layer is going to be 3 * 2  because we hae to dimensional embeddings and we have 3 of them
# and its up to us to decide how many neurons we want inside the layer here we are going with 100 of them
W1 = torch.randn(6,100, generator = g )
b1 = torch.randn(100, generator = g) 
W2 = torch.randn(100, 27, generator = g) #our second layer will take 100 inputs and 
b2 = torch.randn(27, generator = g)
parameters = [C,W1, b1, W2, b2]

In [295]:
sum(p.nelement() for p in parameters) #tells us number of parameters in total

3481

In [296]:
for p in parameters:
    p.requires_grad = True
#we do this because p.requires_grad is false by default but insted of treating these tensors as constant we want pytorch to treat them as variable, Variable which requires to have gradients

In [297]:
for i in range(1000):
    # forward pass
    emb = C[X] #the shape is (32,3,2)
    h = torch.tanh(emb.view(-1,6) @ W1 + b1) #we are using tanh activation fn so the numbers in h will be -1 and 1 
    # We pass -1 to emb.view(), becuase we want pytorch to guess what what will be the right number for rows given we have already told it the number of columns are 6
    # here you can also use emb.reshape(32,6) the difference is emb.view() will not use extra space it ensure that the emb tensor and and the new tensor that we create will use the same data so no memory wastag
    logits = h @ W2 + b2 #the shape is 32,27
    # implementing the loss function
    loss = F.cross_entropy(logits, Y)  #implementing the categorical cross entropy using pytorch
    print(f'loss= {loss.item()} for iteration= {i}') 
    #we use pytorch here because large positive numbers when exponentiated causes the overflow which can be handled by subtracting the biggest number in the logits from each element row wise, and pytorch handles this operation internally

    # Implementing backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # param update
    for p in parameters:
        p.data -= 0.1 * p.grad

loss= 18.32672882080078 for iteration= 0
loss= 16.474193572998047 for iteration= 1
loss= 15.277494430541992 for iteration= 2
loss= 14.292076110839844 for iteration= 3
loss= 13.503080368041992 for iteration= 4
loss= 12.799945831298828 for iteration= 5
loss= 12.176278114318848 for iteration= 6
loss= 11.639505386352539 for iteration= 7
loss= 11.172645568847656 for iteration= 8
loss= 10.7584867477417 for iteration= 9
loss= 10.384634017944336 for iteration= 10
loss= 10.042563438415527 for iteration= 11
loss= 9.726486206054688 for iteration= 12
loss= 9.43183708190918 for iteration= 13
loss= 9.155553817749023 for iteration= 14
loss= 8.895856857299805 for iteration= 15
loss= 8.6515474319458 for iteration= 16
loss= 8.421377182006836 for iteration= 17
loss= 8.203655242919922 for iteration= 18
loss= 7.996348857879639 for iteration= 19
loss= 7.7974138259887695 for iteration= 20
loss= 7.6050639152526855 for iteration= 21
loss= 7.417822360992432 for iteration= 22
loss= 7.234484672546387 for iteratio

In [ ]:
#in th code above it takes 2 minutes to train the model because we are doing batch gradient descent calculating
#loss for every example in the dataset and then updating the weights we can accelarate this by implementing 
#mini batch SGD